In [10]:
!pip install gensim
!pip install pandas
!pip install spacy
!pip install csv
!pip install matplotlib.pyplot
!pip install sklearn

ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv
ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib.pyplot
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [100]:
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import requests
import csv
import gensim
from gensim.models import Word2Vec

In [101]:
file_path = "./arguments-training.tsv"

In [102]:
nlp = spacy.load("en_core_web_sm")

In [103]:
# Lista per salvare le frasi tokenizzate
tok_sents = []

# Estrai e tokenizza la premessa di ciascuna riga nel file TSV
with open(file_path, 'r', encoding='utf-8', newline='') as file:
    reader = csv.DictReader(file, delimiter='\t')
    for row in reader:
        premise_tokens = [token.text.lower() for token in nlp(row['Premise'])]
        tok_sents.append(premise_tokens)

In [104]:
# deleting the missing row
del tok_sents[3358]

In [25]:
url = "https://raw.githubusercontent.com/nicholas-leonard/word2vec/master/questions-words.txt"
test_file = 'questions-words.txt'
questions = requests.get(url).content.decode()
with open(test_file,mode='w',encoding='utf-8') as outputfile:
    outputfile.write(questions)
print(questions[:1000])

: capital-common-countries
Athens Greece Baghdad Iraq
Athens Greece Bangkok Thailand
Athens Greece Beijing China
Athens Greece Berlin Germany
Athens Greece Bern Switzerland
Athens Greece Cairo Egypt
Athens Greece Canberra Australia
Athens Greece Hanoi Vietnam
Athens Greece Havana Cuba
Athens Greece Helsinki Finland
Athens Greece Islamabad Pakistan
Athens Greece Kabul Afghanistan
Athens Greece London England
Athens Greece Madrid Spain
Athens Greece Moscow Russia
Athens Greece Oslo Norway
Athens Greece Ottawa Canada
Athens Greece Paris France
Athens Greece Rome Italy
Athens Greece Stockholm Sweden
Athens Greece Tehran Iran
Athens Greece Tokyo Japan
Baghdad Iraq Bangkok Thailand
Baghdad Iraq Beijing China
Baghdad Iraq Berlin Germany
Baghdad Iraq Bern Switzerland
Baghdad Iraq Cairo Egypt
Baghdad Iraq Canberra Australia
Baghdad Iraq Hanoi Vietnam
Baghdad Iraq Havana Cuba
Baghdad Iraq Helsinki Finland
Baghdad Iraq Islamabad Pakistan
Baghdad Iraq Kabul Afghanistan
Baghdad Iraq London England


Ricerca Parametri per miglior modello Word2Vec

In [16]:
from sklearn.model_selection import ParameterGrid
from gensim.models import Word2Vec
import csv
from tqdm import tqdm
from IPython.display import display, HTML
import time

# Definisco la griglia di ricerca
param_grid = {
    'vector_size': [100, 150, 200],
    'window': [5, 10, 15],
    'min_count': [3, 5, 7],
    'sg': [0, 1],
    'epochs': [10, 20, 50, 100],
    'negative': [10]
}

# Nome del file di output CSV
output_file = 'risultati.csv'

# Scrivo l'intestazione del file CSV
with open(output_file, 'w', newline='') as csvfile:
    fieldnames = ['vector_size', 'window', 'min_count', 'sg', 'epochs', 'negative', 'score']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

# Eseguo la griglia di ricerca con barra di avanzamento
total_models = len(list(ParameterGrid(param_grid)))
for i, params in enumerate(tqdm(ParameterGrid(param_grid), desc='Progresso', total=total_models)):
    w2v_model = Word2Vec(tok_sents, **params)

    # Valuto le prestazioni del modello con un confronto tra analogie
    similarity_score = w2v_model.wv.evaluate_word_analogies(test_file, case_insensitive=True, dummy4unknown=False, similarity_function='most_similar')[0]

    # Salvo i risultati nel file CSV
    with open(output_file, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow({**params, 'score': similarity_score})

    # Aggiorno la barra di avanzamento su Google Colab
    progress = i / total_models * 100
    display(HTML(f'<progress value="{progress}" max="100"></progress>'))

    # Aggiungo un piccolo ritardo per visualizzare l'aggiornamento della barra
    time.sleep(0.1)

print(f"I risultati sono stati scritti su '{output_file}'.")


Progresso:   0%|          | 0/216 [00:00<?, ?it/s]

Progresso:   0%|          | 1/216 [00:04<15:16,  4.26s/it]

Progresso:   1%|          | 2/216 [00:08<14:19,  4.02s/it]

Progresso:   1%|▏         | 3/216 [00:10<12:25,  3.50s/it]

Progresso:   2%|▏         | 4/216 [00:14<12:22,  3.50s/it]

...

Progresso:  99%|█████████▊| 213/216 [2:10:56<06:38, 132.75s/it]

Progresso:  99%|█████████▉| 214/216 [2:12:18<03:55, 117.69s/it]

Progresso: 100%|█████████▉| 215/216 [2:14:26<02:00, 120.69s/it]

Progresso: 100%|██████████| 216/216 [2:17:09<00:00, 38.10s/it] 

I risultati sono stati scritti su 'risultati.csv'.


Creazione Modello Word2Vec

In [127]:
#Creo ed addestro il modello Word2Vec con i miei parametri ideali
w2v_model = Word2Vec(tok_sents, vector_size=150, window=5, min_count=7, sg=0, epochs=50, negative=10)

In [136]:
%store w2v_model

Stored 'w2v_model' (Word2Vec)


In [131]:
w2w_vectors = []
for arg in tok_sents:
    seq = []
    for word in arg:
        if word in w2v_model.wv:
            seq.append(word)
        else:
            pass
    w2w_vectors.append(seq)

In [132]:
len(w2v_vectors)

5392

In [133]:
max_sequence_length = max(len(arg) for arg in tok_sents)
max_sequence_length

147

In [134]:
w2v_model.vector_size

150

In [135]:
%store w2v_vectors

Stored 'w2v_vectors' (list)
